In [1]:
import timeit

from pyspark.sql.dataframe import DataFrame
from pyspark.sql import SparkSession

from ts_train.step.time_bucketing import TimeBucketing
from ts_train.step.aggregation import Aggregation

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/27 17:38:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", "|") \
    .option("encoding", "utf-8") \
    .option("multiline", "False") \
    .csv("../../data/df_ts.csv")

In [4]:
df.show()

+---------------+--------------+----------------+-------------------+-------+-----+--------------+--------+-----------+---------------+------+------+--------------------+--------------------+--------------------+-----+------+-----------+
|ARCA_TIPO_CARTA|DATA_CONTABILE|DATA_TRANSAZIONE|    ORA_TRANSAZIONE|IMPORTO|SEGNO|ID_CLIENTE_BIC|IS_CARTA|TIPO_CANALE|TIPO_CANALE_AGG|IS_BON|IS_SDD|       CATEGORY_LIV0|       CATEGORY_LIV1|       CATEGORY_LIV2|IS_CC|IS_LIB|   MERCHANT|
+---------------+--------------+----------------+-------------------+-------+-----+--------------+--------+-----------+---------------+------+------+--------------------+--------------------+--------------------+-----+------+-----------+
|      EVOLUTION|    2022-01-22|      2022-01-20|2023-07-27 11:25:47|   0.97|    -|    1301589529|    true|WEB_ESTERNO|        DIGITAL| false| false|            shopping|                null|                null|false| false|ali express|
|      EVOLUTION|    2022-01-25|      2022-01-22

In [5]:
df.count()

960404

In [6]:
df_big = df
iterations = 200
for i in range(iterations-1):
    df_big = df_big.union(df)

In [7]:
df_big.count()

23/07/27 17:38:50 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


192080800

In [8]:
time_bucketing = TimeBucketing(
    time_zone="Europe/Rome",
    time_column_name="DATA_TRANSAZIONE",
    time_bucket_size=1,
    time_bucket_granularity="day",
    time_bucket_col_name="bucket",
)

aggregation = Aggregation(
    numerical_col_name=["IMPORTO"],
    identifier_cols_name=["ID_CLIENTE_BIC"],
    all_aggregation_filters=[[("CATEGORY_LIV0", [])]],
    agg_funcs=["sum"],
)

In [9]:
bucketed_df = time_bucketing(df=df_big, spark=spark)

In [10]:
%%timeit

aggregated_df = aggregation(df=bucketed_df, spark=spark)

23/07/27 17:40:08 WARN DAGScheduler: Broadcasting large task binary with size 1036.3 KiB
23/07/27 17:41:49 WARN DAGScheduler: Broadcasting large task binary with size 1036.3 KiB
23/07/27 17:43:29 WARN DAGScheduler: Broadcasting large task binary with size 1036.3 KiB
23/07/27 17:45:06 WARN DAGScheduler: Broadcasting large task binary with size 1036.3 KiB
23/07/27 17:46:54 WARN DAGScheduler: Broadcasting large task binary with size 1036.3 KiB
23/07/27 17:48:56 WARN DAGScheduler: Broadcasting large task binary with size 1036.3 KiB
23/07/27 17:50:48 WARN DAGScheduler: Broadcasting large task binary with size 1036.3 KiB
23/07/27 17:52:31 WARN DAGScheduler: Broadcasting large task binary with size 1036.3 KiB


1min 46s ± 8.28 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 960404 transactions

With precalculated options

907 ms ± 48.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Without options

792 ms ± 39.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

### 192080800 transactions

With precalculated options

1min 46s ± 8.28 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

Without options

2min 33s ± 2.79 s per loop (mean ± std. dev. of 7 runs, 1 loop each)